In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle

In [6]:
data=pd.read_csv('../Churn_Modelling.xls')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
data=data.drop(["RowNumber","CustomerId","Surname"],axis=1)

In [8]:
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [9]:
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])

In [10]:
one_hot_encoder_geo=OneHotEncoder(handle_unknown='ignore')
geo_encoded=one_hot_encoder_geo.fit_transform(data[['Geography']]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoded,columns=one_hot_encoder_geo.get_feature_names_out(['Geography']))

In [11]:
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [12]:
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [13]:
X=data.drop('EstimatedSalary',axis=1)
y=data['EstimatedSalary']

In [14]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [15]:
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [16]:
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('one_hot_encoder_geo.pkl','wb') as file:
    pickle.dump(one_hot_encoder_geo,file)

with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [20]:
model=Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32,activation='relu'),
    Dense(1)
])

2024-12-15 21:08:47.203841: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-12-15 21:08:47.207477: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [21]:
model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

log_dir='logs/fit/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorboard_callbacks = TensorBoard(log_dir=log_dir,histogram_freq=1)

In [23]:
early_stopping=EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [24]:
history=model.fit(
    X_train,y_train,validation_data=(X_test,y_test),
    epochs=100,
    callbacks=[early_stopping,tensorboard_callbacks]
)

Epoch 1/100
250/250 [==============================] - 2s 5ms/step - loss: 100382.2812 - mae: 100382.2812 - val_loss: 98533.2812 - val_mae: 98533.2812
Epoch 2/100
250/250 [==============================] - 1s 3ms/step - loss: 99699.2188 - mae: 99699.2188 - val_loss: 97148.5391 - val_mae: 97148.5391
Epoch 3/100
250/250 [==============================] - 1s 3ms/step - loss: 97301.1953 - mae: 97301.1953 - val_loss: 93607.5156 - val_mae: 93607.5156
Epoch 4/100
250/250 [==============================] - 1s 3ms/step - loss: 92536.7109 - mae: 92536.7109 - val_loss: 87604.8047 - val_mae: 87604.8047
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 85508.0312 - mae: 85508.0312 - val_loss: 79706.4844 - val_mae: 79706.4844
Epoch 6/100
250/250 [==============================] - 1s 3ms/step - loss: 77012.7891 - mae: 77012.7891 - val_loss: 71209.9688 - val_mae: 71209.9688
Epoch 7/100
250/250 [==============================] - 1s 3ms/step - loss: 68483.3516 - mae: 68483.3516 

In [25]:
%load_ext tensorboard

In [41]:
%tensorboard --logdir logs/fit/20241215-211859

Reusing TensorBoard on port 6007 (pid 7031), started 0:01:25 ago. (Use '!kill 7031' to kill it.)

In [42]:
test_loss,test_mae=model.evaluate(X_test,y_test)

63/63 [==============================] - 0s 2ms/step - loss: 50126.3555 - mae: 50126.3555


In [44]:
print(f'test MAE {test_mae}')

test MAE 50126.35546875


In [45]:
model.save('regressionModel.h5')

/home/krishsharma/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
